In [1]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp
import cmath
import time

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 
from scipy.linalg import expm

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize
from random import randint, uniform, random

In [2]:
# Manipulação das matrizes, soma e subtração

def somar(A, B):
    C = []
    num_linhas_a = len(A)
    num_colunas_a = len(A[0])
    
    for i in range (num_linhas_a):
        linha = [0]*num_colunas_a
        C.append(linha)
        for j in range(num_colunas_a):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    num_linhas_a = len(A)
    num_colunas_a = len(A[0])
    
    for i in range (num_linhas_a):
        linha = [0]*num_colunas_a
        C.append(linha)
        for j in range(num_linhas_a):
            C[i][j] = A[i][j] - B[i][j]

    return C

In [3]:
## CONSTANTES PARA O CONTROLADOR

# Considerações: constante de Planck verdadeira: 1, massa: 1

TEMPO_ANALISE = 10 
QUANTIDADE_PONTOS_AMOSTRAGEM = 1000
PASSO = TEMPO_ANALISE/QUANTIDADE_PONTOS_AMOSTRAGEM

MIN_HORIZONTE = 3 # Horizonte mínimo da análise
MAX_HORIZONTE = 30 # Horizonte máximo da análise

# Com horizonte em 10, a avaliação dura em torno de 80 minutos 

DESEJADO = [[complex(0.0,0.0)],[complex(1.0,0.0)]] # [[c1];[c2]] = [[0.707],[-0.707]]
RHO_DESEJADO = np.matmul(DESEJADO, np.transpose(DESEJADO))

# Determinação dos valores de início e fim da análise
num = PASSO
contador = 0
while num < 1:
    num *= 10
    contador += 1

EXPOENTE = contador
INICIO_ANALISE = 0*10**(-EXPOENTE) # Tempo inicial da analise em um horizonte
FINAL_ANALISE = (1*10**(-EXPOENTE))+PASSO # Tempo final da analise em um horizonte

In [4]:
## CONSTANTES PARA O ALGORITMO GENÉTICO

ELITISMO = True
PORCENTAGEM_ELITISMO = 0.20

TAMANHO_POP = 50
TAXA_MUTACAO = 0.10
TAXA_CRUZAMENTO = 0.70
GERACOES = 100

# Valores mínimos e máximos para gerar uma população
MIN = -5
MAX = 5

In [5]:
def dpsi_dt(t, ro, H): # A derivada da ro em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    A = (np.dot(H,ro))-(np.dot(ro,H)) # [H,p]
    return -1j*A

# Runge-Kutta de quarta ordem

def runge_kutta(ro, fator_runge_kutta, hamiltoniano, tempo_inicial = 0):

    k1 = dpsi_dt(tempo_inicial, ro, hamiltoniano)
    k2 = dpsi_dt(tempo_inicial + 0.5 * fator_runge_kutta, ro + 0.5*fator_runge_kutta*k1, hamiltoniano)
    k3 = dpsi_dt(tempo_inicial + 0.5 * fator_runge_kutta, ro + 0.5*fator_runge_kutta*k2, hamiltoniano)
    k4 = dpsi_dt(tempo_inicial + fator_runge_kutta, ro + 0.5*fator_runge_kutta*k3, hamiltoniano)
    
    ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
    
    A = (np.dot(2,k3) + k4)
    B = (np.dot(2,k2) + k1)
    C = (A + B)

    runge = ro + np.dot((fator_runge_kutta / 6.0),(C))
    
    return runge

In [6]:
def func_objetivo(vetor_controles, tempo_atual, rho_desejado, 
                  rho_atual, H, horizonte, fator_runge_kutta):
    
    fo = 0
    
    tempo_inicial = INICIO_ANALISE
    tempo_final = FINAL_ANALISE
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, vetor_controles[0]], [vetor_controles[0], 0]]
    
    matriz_inicial = rho_atual

    matriz_desejada = np.zeros((2,2),dtype=np.complex_)

    R = np.dot(np.dot(-1j,H),tempo_atual)
    U = expm(R)
    U = np.asmatrix(U)
    U_dagger = U.getH()
    matriz_desejada = U*rho_desejado*U_dagger

    fo += (np.linalg.norm(matriz_inicial-matriz_desejada))**2

    runge = runge_kutta(matriz_inicial, fator_runge_kutta, 
                        somar(H, controles), tempo_inicial = tempo_inicial)

    matriz_inicial = runge

    tempo_inicial = round(tempo_inicial+PASSO, EXPOENTE)
    tempo_final = round(tempo_final+PASSO, EXPOENTE)
    tempo = round(tempo_atual+PASSO, EXPOENTE)

    while horizonte > 1:
        
        R = np.dot(np.dot(-1j,H),tempo_atual)
        U = expm(R)
        U = np.asmatrix(U)
        U_dagger = U.getH()
        matriz_desejada = U*rho_desejado*U_dagger
        
        fo += (np.linalg.norm(matriz_inicial-matriz_desejada))**2
        
        controles = [[0, vetor_controles[len(vetor_controles)-horizonte+1]], 
                     [vetor_controles[len(vetor_controles)-horizonte+1], 0]]

        runge = runge_kutta(matriz_inicial, fator_runge_kutta, 
                        somar(H, controles), tempo_inicial = tempo_inicial)

        matriz_inicial = runge

        tempo_inicial = round(tempo_inicial+PASSO, EXPOENTE)
        tempo_final = round(tempo_final+PASSO, EXPOENTE)
        tempo = round(tempo_atual+PASSO, EXPOENTE)
        horizonte -= 1

    return fo

In [7]:
def main(horizonte, rho_inicial):
    
    u = [] # Controladores iniciais
    u.append(uniform(-5,5))
    while len(u) < horizonte:
        u.append(u[0])
        
    # O que será retornado por essa função
    
    vetor_controles = np.zeros((1, QUANTIDADE_PONTOS_AMOSTRAGEM), dtype=np.complex_) # Valores de controle
    constantes = np.zeros((2, QUANTIDADE_PONTOS_AMOSTRAGEM), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
    valores_onda = np.zeros((2, QUANTIDADE_PONTOS_AMOSTRAGEM), dtype=np.complex_)
    
    H = np.zeros((2,2), dtype=np.complex_)
    H = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    tempo = 0
    pos = 0
    while tempo < TEMPO_ANALISE:

        print(rho_inicial)
        # Variação do Psi
        valores_onda[0][pos] = rho_inicial[0][0]
        valores_onda[1][pos] = rho_inicial[1][1]
        
        # Obtenção das constantes que multiplicam Psi
        # C_0 = Psi_0/[e^(1/2*pi*tempo)] equação 2.18 Griffiths
        constantes[0][pos] = rho_inicial[0][0]/np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
        constantes[1][pos] = rho_inicial[1][1]/np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)
        
        ## Minimização
    
        resultado = optimize.minimize(func_objetivo, u, args=(tempo, RHO_DESEJADO, rho_inicial, H, horizonte, PASSO), method='CG')
        
        controle = np.zeros((2,2), dtype=np.complex_)
        controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
        H = somar(H, controle)
    
        ## Adaptação da onda ao controle
        
        rho_avancado = runge_kutta(rho_inicial, PASSO, H, tempo_inicial = INICIO_ANALISE)
        rho_inicial = rho_avancado
        
        # Reset
        H = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
        u = [] # Controladores iniciais
        while len(u) < horizonte:
            u.append(resultado.x[0])
        
        pos += 1
        tempo = round(tempo+PASSO, EXPOENTE)
    

In [8]:
PSI = np.ones((2,1), dtype=np.complex_)
PSI = [[0.80],[0.60]]
ro = np.matmul(PSI, np.transpose(PSI))
main(4, ro)

[[0.64 0.48]
 [0.48 0.36]]
[[0.35127975+0.j         0.47738116-0.24979763j]
 [0.47738116+0.24979763j 0.64872025+0.j        ]]
[[0.5256632 +0.j         0.47917778+0.33926962j]
 [0.47917778-0.33926962j 0.4743368 +0.j        ]]
[[0.43728257+0.j         0.47818179-0.33001354j]
 [0.47818179+0.33001354j 0.56271743+0.j        ]]
[[0.41791004+0.j         0.47798019+0.37638529j]
 [0.47798019-0.37638529j 0.58208996+0.j        ]]
[[0.47557435+0.j         0.47865114-0.36851986j]
 [0.47865114+0.36851986j 0.52442565+0.j        ]]
[[0.38382676+0.j         0.47768792+0.39778916j]
 [0.47768792-0.39778916j 0.61617324+0.j        ]]
[[0.48691607+0.j         0.47889646-0.39542938j]
 [0.47889646+0.39542938j 0.51308393+0.j        ]]
[[0.37714697+0.j         0.47773775+0.42020932j]
 [0.47773775-0.42020932j 0.62285303+0.j        ]]
[[0.4865967 +0.j         0.47902104-0.41781307j]
 [0.47902104+0.41781307j 0.5134033 +0.j        ]]
[[0.38105952+0.j         0.47790279+0.44237172j]
 [0.47790279-0.44237172j 0.618940